In [1]:
import pickle
import numpy as np
import neuraltoolkit as ntk
import glob
import csv
import os
import smart_open
import boto3
import tqdm
import argparse
import fnmatch
import io
import re
import tempfile
from pathlib import Path
import pandas as pd

In [2]:
def get_remote_files(prefix: str):
    try:
        response = client.list_objects_v2(Bucket="hengenlab", Prefix=prefix)["Contents"]
    except:
        response = []
    return ntk.ntk_videos.natural_sort([file["Key"] for file in response])

def get_s3_client():
    with open(f"{Path.home()}/.aws/credentials", "r") as f:
        for line in f:
            if "aws_access_key_id" in line:
                id = re.search(r"\W(\w+)", line).group(1)
            if "aws_secret_access_key" in line:
                key = re.search(r"\W(\w+)", line).group(1)
        return boto3.Session(aws_access_key_id=id, aws_secret_access_key=key).client(
            "s3", endpoint_url="https://s3-central.nrp-nautilus.io"
        )

def get_animal_file(animal, client):
    contents = client.list_objects_v2(Bucket="hengenlab", Prefix=f'{animal}/Runs/')['Contents']
    files = [content['Key'] for content in contents if fnmatch.fnmatch(content['Key'], '*.txt')]
    return files


- Use this file to check the CNN results

First, initialize a client and obtain the prefix to the wanted files.

In [3]:
ANIMAL = 'CAF62_day1'

client = get_s3_client()
summaries = get_animal_file(ANIMAL, client)

for summary in summaries:
    with smart_open.open(
        's3://hengenlab/'+summary, 'r', transport_params=dict(client=client)
    ) as f:
        print(f.read(375))

KeyError: 'Contents'

In [52]:
print(f.read())


per-label-train-set (All Samples) (0 samples)
    NaN: WAKE            (accuracy) (      0 samples)
    NaN: NREM            (accuracy) (      0 samples)
    NaN: REM             (accuracy) (      0 samples)
   0.00: F1 Multiclass  (avg=micro) (      0 samples)
    NaN: F1 Multiclass  (avg=macro) (      0 samples)
    NaN: Balanced Accuracy Score    (      0 samples)

per-video-metrics (All Samples)
                                   Passive/   Micro-
    WAKE   |   NREM   |   REM    |  Active  |  arousal | video-filename
  -------- | -------- | -------- | -------- | -------- | ---------------
    0.72   |   0.05   |   0.48   |    NaN   |    NaN   | dummy (test-set)


per-label-test-set (Between State Change) (1187963 samples)
   0.72: WAKE            (accuracy) ( 970935 samples)
   0.05: NREM            (accuracy) ( 185538 samples)
   0.48: REM             (accuracy) (  31490 samples)
   0.61: F1 Multiclass  (avg=micro) (1187963 samples)
   0.32: F1 Multiclass  (avg=macro) (1187963 s

In [16]:
with smart_open.open(
    's3://hengenlab/CAF62_day1/Labels/labels_sleepstate_v2.1_CAF62_day1.npz',
    'rb',
    transport_params=dict(client=client)) as f:
    f1 = np.load(f)

In [17]:
f1.files

['labels_matrix', 'video_files', 'neural_files']

In [19]:
f1['video_files']

array(['dummy'], dtype='<U5')